In [1]:
!pip install flair

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for flair from https://files.pythonhosted.org/packages/af/16/536683088c7306bc51cc3cc58605759ebd83b3f7ffd05a9399f4b99c8614/flair-0.13.1-py3-none-any.whl.metadata
  Using cached bpemb-0.3.4-py3-none-any.whl (19 kB)
  Obtaining dependency information for conllu>=4.0 from https://files.pythonhosted.org/packages/ce/3f/70a1dc5bc536755ec082b806594598a10cfffaf0de978f51d4e0e4fdfa47/conllu-4.5.3-py2.py3-none-any.whl.metadata
  Obtaining dependency information for deprecated>=1.2.13 from https://files.pythonhosted.org/packages/20/8d/778b7d51b981a96554f29136cd59ca7880bf58094338085bcf2a979a0e6a/Deprecated-1.2.14-py2.py3-none-any.whl.metadata
  Obtaining dependency information for ftfy>=6.1.0 from https://files.pythonhosted.org/packages/91/f8/dfa32d06cfcbdb76bc46e0f5d69c537de33f4cedb1a15cd4746ab45a6a26/ftfy-6.1.3-py3-none-any.whl.metadata
  Obtaining dependency information for gdown>=4.4.

In [9]:
from flair.data import Sentence
from flair.nn import Classifier
import pandas as pd

In [10]:
sample = pd.read_csv("../../data/sampling/FAA_sample_100.csv")

In [16]:
out_dict = {'index':[], 'c5_unique_id':[], 'c119_text':[], 'flair_ner_recd_ents':[], 'flair_label':[], 'eval':[]}

tagger = Classifier.load('ner')

for index in range(len(sample)):
    text = sample['c119'][index]
    sentence = Sentence(text)
    tagger.predict(sentence)
    sent_dict = sentence.to_dict()
    if 'entities' not in sent_dict:
        print("err on ", sent_dict)
        continue
    ents = sent_dict['entities']
    
    if len(ents) == 0:
        ents = [{"text":"", "labels":""}]     # still want to create an entry in the final sheet, but leave entity column empty
    for ent in ents:
        out_dict['index'].append(sample['Unnamed: 0'][index])
        out_dict['c5_unique_id'].append(sample['c5'][index])
        out_dict['c119_text'].append(sample['c119'][index])
        out_dict['flair_ner_recd_ents'].append(ent['text'])
        out_dict['flair_label'].append(ent['labels'])
        out_dict['eval'].append("")

2024-02-16 13:51:29,017 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [17]:
pd.DataFrame(out_dict).to_excel("flair_ner.xlsx", index=False)